# 🔍 Detecção de Fraude em Cartões de Crédito

## Projeto Completo de Machine Learning

Este notebook implementa um sistema de detecção de fraude utilizando:
- **Modelos Supervisionados**: Logistic Regression, Random Forest, XGBoost, LightGBM
- **Detecção de Anomalias**: Isolation Forest
- **Abordagem Híbrida**: Combinação de modelos para máxima cobertura

### Dataset
Credit Card Fraud Detection (Kaggle) - 284.807 transações com 492 fraudes (~0.17%)

---

## 1. Setup e Configuração

In [ ]:
# Instalação de dependências (executar se necessário)
# !pip install pandas numpy scikit-learn xgboost lightgbm imbalanced-learn matplotlib seaborn joblib

In [ ]:
import sys
import os
from pathlib import Path

# Adicionar src ao path
PROJECT_ROOT = Path.cwd()
if str(PROJECT_ROOT / 'src') not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT / 'src'))

# Imports padrão
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

warnings.filterwarnings('ignore')

# Imports do projeto
from data import (
    load_credit_card_data, get_data_summary, print_data_summary,
    split_features_target, create_time_features, create_amount_features
)
from features import (
    FeatureEngineer, prepare_train_test_split, handle_imbalance,
    create_interaction_features, get_feature_statistics
)
from models import (
    FraudClassifier, train_multiple_models, hyperparameter_tuning,
    get_model_summary, AnomalyDetector, HybridFraudDetector, train_anomaly_detector
)
from evaluation import (
    calculate_metrics, print_evaluation_report, compare_models,
    find_optimal_threshold, calculate_business_metrics, ModelEvaluator,
    plot_class_distribution, plot_feature_distributions, plot_confusion_matrix,
    plot_roc_curves, plot_precision_recall_curves, plot_feature_importance,
    plot_model_comparison, plot_threshold_analysis
)
from utils import set_seed, print_section, Timer, print_dependencies_status

# Configurações
set_seed(42)
pd.set_option('display.max_columns', 50)
pd.set_option('display.float_format', '{:.4f}'.format)

print("✓ Setup concluído!")
print_dependencies_status()

## 2. Carregamento dos Dados

### Download do Dataset
Baixe o dataset do Kaggle: [Credit Card Fraud Detection](https://www.kaggle.com/mlg-ulb/creditcardfraud)

Coloque o arquivo `creditcard.csv` na pasta `data/raw/`

In [ ]:
# Criar diretórios se não existirem
DATA_DIR = PROJECT_ROOT / 'data' / 'raw'
DATA_DIR.mkdir(parents=True, exist_ok=True)

DATA_PATH = DATA_DIR / 'creditcard.csv'

# Verificar se o dataset existe
if not DATA_PATH.exists():
    print(f"⚠️  Dataset não encontrado em: {DATA_PATH}")
    print("\nBaixe o dataset do Kaggle:")
    print("https://www.kaggle.com/mlg-ulb/creditcardfraud")
    print(f"\nColoque o arquivo creditcard.csv em: {DATA_DIR}")
else:
    print(f"✓ Dataset encontrado: {DATA_PATH}")

In [ ]:
# Carregar dados
with Timer("Carregamento dos dados"):
    df = load_credit_card_data(DATA_PATH)

# Resumo do dataset
summary = get_data_summary(df)
print_data_summary(summary)

In [ ]:
# Visualizar primeiras linhas
df.head()

In [ ]:
# Informações do DataFrame
df.info()

In [ ]:
# Estatísticas descritivas
df.describe()

## 3. Análise Exploratória dos Dados (EDA)

In [ ]:
print_section("ANÁLISE EXPLORATÓRIA DOS DADOS")

### 3.1 Distribuição das Classes

In [ ]:
fig = plot_class_distribution(df['Class'])
plt.show()

**Observação**: O dataset é extremamente desbalanceado (0.17% de fraudes). Isso requer técnicas especiais de balanceamento e métricas apropriadas para avaliação.

### 3.2 Análise da Variável Amount

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Distribuição de Amount por classe
for label, color, name in [(0, '#2ecc71', 'Normal'), (1, '#e74c3c', 'Fraude')]:
    data = df[df['Class'] == label]['Amount']
    axes[0].hist(data, bins=50, alpha=0.6, color=color, label=name, density=True)

axes[0].set_xlabel('Amount')
axes[0].set_ylabel('Densidade')
axes[0].set_title('Distribuição de Amount por Classe')
axes[0].legend()
axes[0].set_xlim(0, 500)  # Limitar para melhor visualização

# Boxplot
df.boxplot(column='Amount', by='Class', ax=axes[1])
axes[1].set_title('Amount por Classe')
axes[1].set_xlabel('Classe (0=Normal, 1=Fraude)')
axes[1].set_ylabel('Amount')
plt.suptitle('')

plt.tight_layout()
plt.show()

# Estatísticas
print("\nEstatísticas de Amount por Classe:")
print(df.groupby('Class')['Amount'].describe())

### 3.3 Análise Temporal

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Distribuição de Time por classe
for label, color, name in [(0, '#2ecc71', 'Normal'), (1, '#e74c3c', 'Fraude')]:
    data = df[df['Class'] == label]['Time']
    axes[0].hist(data, bins=50, alpha=0.6, color=color, label=name, density=True)

axes[0].set_xlabel('Time (segundos)')
axes[0].set_ylabel('Densidade')
axes[0].set_title('Distribuição Temporal das Transações')
axes[0].legend()

# Converter para horas e ver padrão
df['Hour'] = (df['Time'] / 3600) % 24
fraud_by_hour = df.groupby(['Hour', 'Class']).size().unstack(fill_value=0)
fraud_by_hour['fraud_rate'] = fraud_by_hour[1] / (fraud_by_hour[0] + fraud_by_hour[1]) * 100

axes[1].plot(fraud_by_hour.index, fraud_by_hour['fraud_rate'], 'r-', linewidth=2)
axes[1].set_xlabel('Hora do Dia')
axes[1].set_ylabel('Taxa de Fraude (%)')
axes[1].set_title('Taxa de Fraude por Hora')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

### 3.4 Análise das Features PCA (V1-V28)

In [ ]:
# Calcular estatísticas por classe para cada feature
pca_features = [f'V{i}' for i in range(1, 29)]
X, y = split_features_target(df)
feature_stats = get_feature_statistics(X[pca_features], y)

print("Top 10 Features com maior diferença entre classes:")
feature_stats.head(10)

In [ ]:
# Plotar as 8 features mais discriminativas
top_features = feature_stats.head(8)['feature'].tolist()
fig = plot_feature_distributions(df, top_features, n_cols=4, figsize=(16, 8))
plt.show()

**Insights da EDA:**
1. Dataset extremamente desbalanceado (0.17% fraudes)
2. Fraudes têm valores médios mais baixos que transações normais
3. Algumas features PCA (V14, V12, V10, V17) mostram clara separação entre classes
4. Padrões temporais podem existir, mas são sutis

## 4. Engenharia de Features

In [ ]:
print_section("ENGENHARIA DE FEATURES")

In [ ]:
# Criar features temporais
df_processed = create_time_features(df)
print("✓ Features temporais criadas: Hour, Hour_sin, Hour_cos, Is_Night")

# Criar features de valor
df_processed = create_amount_features(df_processed)
print("✓ Features de valor criadas: Amount_log, Amount_category, Is_High_Amount")

# Criar features de interação
df_processed = create_interaction_features(df_processed)
print("✓ Features de interação criadas: V1_V2, V3_V4, PCA_magnitude")

print(f"\nTotal de features: {df_processed.shape[1] - 1}")

In [ ]:
# Separar features e target
X, y = split_features_target(df_processed, drop_cols=['Time', 'Hour'])
print(f"Shape X: {X.shape}")
print(f"Shape y: {y.shape}")
print(f"\nFeatures: {list(X.columns)}")

In [ ]:
# Dividir em treino e teste
X_train, X_test, y_train, y_test = prepare_train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=True
)

print(f"Conjunto de treino: {X_train.shape[0]:,} amostras")
print(f"Conjunto de teste: {X_test.shape[0]:,} amostras")
print(f"\nDistribuição no treino: {dict(y_train.value_counts())}")
print(f"Distribuição no teste: {dict(y_test.value_counts())}")

In [ ]:
# Escalar features
feature_engineer = FeatureEngineer(scaler_type='robust')
X_train_scaled = feature_engineer.fit_transform(X_train)
X_test_scaled = feature_engineer.transform(X_test)

print("✓ Features escaladas com RobustScaler")

## 5. Treinamento dos Modelos Supervisionados

In [ ]:
print_section("TREINAMENTO DOS MODELOS")

### 5.1 Configuração dos Modelos

Vamos treinar e comparar 4 modelos:

1. **Logistic Regression**: Baseline simples e interpretável
2. **Random Forest**: Ensemble de árvores, robusto a overfitting
3. **XGBoost**: Gradient boosting otimizado, excelente para dados tabulares
4. **LightGBM**: Gradient boosting eficiente, bom para grandes datasets

In [ ]:
# Configuração dos modelos
# Nota: scale_pos_weight para XGBoost é calculado como ratio das classes
scale_pos_weight = (y_train == 0).sum() / (y_train == 1).sum()
print(f"Scale pos weight (XGBoost): {scale_pos_weight:.2f}")

model_configs = {
    'Logistic Regression': {
        'type': 'logistic_regression',
        'params': {
            'max_iter': 1000,
            'class_weight': 'balanced',
            'random_state': 42,
            'solver': 'lbfgs'
        }
    },
    'Random Forest': {
        'type': 'random_forest',
        'params': {
            'n_estimators': 100,
            'max_depth': 10,
            'class_weight': 'balanced',
            'random_state': 42,
            'n_jobs': -1
        }
    },
    'XGBoost': {
        'type': 'xgboost',
        'params': {
            'n_estimators': 100,
            'max_depth': 5,
            'learning_rate': 0.1,
            'scale_pos_weight': scale_pos_weight,
            'random_state': 42,
            'n_jobs': -1,
            'eval_metric': 'aucpr',
            'use_label_encoder': False
        }
    },
    'LightGBM': {
        'type': 'lightgbm',
        'params': {
            'n_estimators': 100,
            'max_depth': 5,
            'learning_rate': 0.1,
            'class_weight': 'balanced',
            'random_state': 42,
            'n_jobs': -1,
            'verbose': -1
        }
    }
}

In [ ]:
# Treinar todos os modelos
with Timer("Treinamento de todos os modelos"):
    models = train_multiple_models(X_train_scaled, y_train, model_configs)

In [ ]:
# Resumo dos modelos treinados
model_summary = get_model_summary(models)
model_summary

## 6. Avaliação dos Modelos

In [ ]:
print_section("AVALIAÇÃO DOS MODELOS")

In [ ]:
# Criar avaliador
evaluator = ModelEvaluator(y_test.values, amounts=X_test['Amount'].values)

# Avaliar cada modelo
results = {}
predictions = {}
probabilities = {}

for name, model in models.items():
    y_pred = model.predict(X_test_scaled)
    y_proba = model.predict_proba(X_test_scaled)[:, 1]
    
    predictions[name] = y_pred
    probabilities[name] = y_proba
    
    results[name] = evaluator.evaluate_model(name, y_pred, y_proba)

In [ ]:
# Tabela comparativa
print_section("COMPARAÇÃO DE MODELOS")

comparison_df = evaluator.get_comparison_table()
comparison_df

### 6.1 Visualização das Curvas ROC

In [ ]:
fig = plot_roc_curves(y_test.values, probabilities)
plt.show()

### 6.2 Visualização das Curvas Precision-Recall

**Importante**: Para datasets desbalanceados, a curva PR é mais informativa que ROC.

In [ ]:
fig = plot_precision_recall_curves(y_test.values, probabilities)
plt.show()

### 6.3 Comparação Visual dos Modelos

In [ ]:
fig = plot_model_comparison(
    comparison_df, 
    metrics=['precision', 'recall', 'f1', 'average_precision']
)
plt.show()

### 6.4 Matriz de Confusão do Melhor Modelo

In [ ]:
# Identificar melhor modelo por Average Precision
best_model_name = comparison_df['average_precision'].idxmax()
print(f"Melhor modelo (Average Precision): {best_model_name}")

fig = plot_confusion_matrix(
    y_test.values, 
    predictions[best_model_name],
    title=f"Matriz de Confusão - {best_model_name}"
)
plt.show()

### 6.5 Importância das Features

In [ ]:
# Importância das features do melhor modelo baseado em árvores
for model_name in ['XGBoost', 'Random Forest', 'LightGBM']:
    if model_name in models:
        importance = models[model_name].get_feature_importance()
        if importance:
            fig = plot_feature_importance(
                importance, 
                top_n=15,
                title=f"Importância das Features - {model_name}"
            )
            plt.show()
            break

## 7. Análise de Threshold

In [ ]:
print_section("ANÁLISE DE THRESHOLD")

In [ ]:
# Análise de threshold para o melhor modelo
best_proba = probabilities[best_model_name]

fig = plot_threshold_analysis(y_test.values, best_proba)
plt.show()

In [ ]:
# Encontrar threshold ótimo
optimal_threshold, optimal_f1 = find_optimal_threshold(y_test.values, best_proba, metric='f1')
print(f"Threshold ótimo para F1: {optimal_threshold:.2f}")
print(f"F1-Score no threshold ótimo: {optimal_f1:.4f}")

# Comparar com threshold padrão
y_pred_default = (best_proba >= 0.5).astype(int)
y_pred_optimal = (best_proba >= optimal_threshold).astype(int)

print("\nComparação de resultados:")
print(f"\nThreshold 0.5:")
metrics_default = calculate_metrics(y_test.values, y_pred_default, best_proba)
print(f"  Precision: {metrics_default['precision']:.4f}")
print(f"  Recall: {metrics_default['recall']:.4f}")
print(f"  F1: {metrics_default['f1']:.4f}")

print(f"\nThreshold {optimal_threshold:.2f}:")
metrics_optimal = calculate_metrics(y_test.values, y_pred_optimal, best_proba)
print(f"  Precision: {metrics_optimal['precision']:.4f}")
print(f"  Recall: {metrics_optimal['recall']:.4f}")
print(f"  F1: {metrics_optimal['f1']:.4f}")

## 8. Detecção de Anomalias (Não Supervisionado)

In [ ]:
print_section("DETECÇÃO DE ANOMALIAS")

### 8.1 Isolation Forest

O Isolation Forest aprende o comportamento "normal" e detecta outliers. É útil para identificar novos padrões de fraude não vistos no treinamento supervisionado.

In [ ]:
# Treinar Isolation Forest apenas com dados normais
anomaly_detector = train_anomaly_detector(
    X_train_scaled, 
    y_train,
    model_type='isolation_forest',
    params={
        'n_estimators': 100,
        'contamination': 0.001,  # ~0.1% de anomalias esperadas
        'random_state': 42,
        'n_jobs': -1
    },
    train_on_normal_only=True
)

In [ ]:
# Avaliar detector de anomalias
anomaly_predictions = anomaly_detector.predict(X_test_scaled)
# Converter: -1 (anomalia) -> 1 (fraude), 1 (normal) -> 0
anomaly_as_fraud = (anomaly_predictions == -1).astype(int)

print("Avaliação do Isolation Forest:")
anomaly_metrics = print_evaluation_report(
    y_test.values, 
    anomaly_as_fraud, 
    anomaly_detector.predict_proba_anomaly(X_test_scaled),
    "Isolation Forest"
)

## 9. Sistema Híbrido

In [ ]:
print_section("SISTEMA HÍBRIDO")

O sistema híbrido combina:
- **Modelo Supervisionado**: Detecta padrões conhecidos de fraude
- **Detector de Anomalias**: Identifica comportamentos atípicos (possivelmente novas fraudes)

Uma transação é marcada como suspeita se qualquer um dos métodos indicar alto risco.

In [ ]:
# Criar detector híbrido
best_supervised_model = models[best_model_name]

hybrid_detector = HybridFraudDetector(
    supervised_model=best_supervised_model,
    anomaly_detector=anomaly_detector,
    supervised_weight=0.7,  # 70% peso para modelo supervisionado
    anomaly_weight=0.3      # 30% peso para detector de anomalias
)

print(f"Sistema híbrido criado com:")
print(f"  - Modelo supervisionado: {best_model_name} (peso: 70%)")
print(f"  - Detector de anomalias: Isolation Forest (peso: 30%)")

In [ ]:
# Avaliar sistema híbrido
hybrid_proba = hybrid_detector.predict_proba(X_test_scaled)
hybrid_pred = hybrid_detector.predict(X_test_scaled, threshold=0.5)

print("Avaliação do Sistema Híbrido:")
hybrid_metrics = print_evaluation_report(
    y_test.values, 
    hybrid_pred, 
    hybrid_proba,
    "Sistema Híbrido"
)

In [ ]:
# Análise detalhada do sistema híbrido
breakdown = hybrid_detector.get_detection_breakdown(X_test_scaled)
breakdown_fraud = breakdown[y_test.values == 1]

print("\nAnálise das fraudes detectadas:")
print(f"Total de fraudes: {len(breakdown_fraud)}")
print(f"\nFraudes detectadas pelo supervisionado (proba > 0.5): {(breakdown_fraud['supervised_proba'] > 0.5).sum()}")
print(f"Fraudes detectadas pelo detector de anomalias (proba > 0.5): {(breakdown_fraud['anomaly_proba'] > 0.5).sum()}")
print(f"Fraudes detectadas pelo sistema híbrido: {breakdown_fraud['final_prediction'].sum()}")

## 10. Comparação Final e Conclusões

In [ ]:
print_section("COMPARAÇÃO FINAL")

In [ ]:
# Adicionar resultados do Isolation Forest e Sistema Híbrido à comparação
all_results = results.copy()
all_results['Isolation Forest'] = anomaly_metrics
all_results['Sistema Híbrido'] = hybrid_metrics

# Criar DataFrame de comparação
final_comparison = compare_models(all_results)
final_comparison = final_comparison.sort_values('average_precision', ascending=False)

print("Ranking dos Modelos (por Average Precision):")
final_comparison

In [ ]:
# Visualização final
fig = plot_model_comparison(
    final_comparison, 
    metrics=['precision', 'recall', 'f1', 'average_precision'],
    figsize=(14, 6)
)
plt.show()

In [ ]:
# Curvas ROC e PR incluindo todos os modelos
all_probabilities = probabilities.copy()
all_probabilities['Isolation Forest'] = anomaly_detector.predict_proba_anomaly(X_test_scaled)
all_probabilities['Sistema Híbrido'] = hybrid_proba

fig = plot_precision_recall_curves(y_test.values, all_probabilities, figsize=(12, 8))
plt.show()

## 11. Conclusões e Recomendações

In [ ]:
print_section("CONCLUSÕES E RECOMENDAÇÕES")

In [ ]:
# Identificar melhor modelo
best_overall = final_comparison['average_precision'].idxmax()
best_recall = final_comparison['recall'].idxmax()
best_precision = final_comparison['precision'].idxmax()

print("RESULTADOS FINAIS")
print("="*60)
print(f"\n1. MELHOR MODELO GERAL (Average Precision): {best_overall}")
print(f"   - Average Precision: {final_comparison.loc[best_overall, 'average_precision']:.4f}")
print(f"   - F1-Score: {final_comparison.loc[best_overall, 'f1']:.4f}")

print(f"\n2. MELHOR RECALL (detectar mais fraudes): {best_recall}")
print(f"   - Recall: {final_comparison.loc[best_recall, 'recall']:.4f}")

print(f"\n3. MELHOR PRECISION (menos falsos positivos): {best_precision}")
print(f"   - Precision: {final_comparison.loc[best_precision, 'precision']:.4f}")

print("\n" + "="*60)
print("RECOMENDAÇÕES")
print("="*60)
print("""
1. PRODUÇÃO:
   - Use o modelo com melhor Average Precision como modelo principal
   - Ajuste o threshold conforme a tolerância a falsos positivos
   - Considere o Sistema Híbrido para detectar fraudes emergentes

2. MONITORAMENTO:
   - Implemente monitoramento contínuo das métricas
   - Retreine periodicamente com novos dados
   - Monitore drift nos padrões de fraude

3. MELHORIAS FUTURAS:
   - Adicionar mais features de comportamento do usuário
   - Implementar modelo de séries temporais para padrões sequenciais
   - Considerar ensemble de múltiplos modelos
   - Testar deep learning (autoencoders) para detecção de anomalias
""")

## 12. Salvar Modelos para Deploy

In [ ]:
# Criar diretório para modelos
MODELS_DIR = PROJECT_ROOT / 'models_saved'
MODELS_DIR.mkdir(parents=True, exist_ok=True)

# Salvar melhor modelo supervisionado
best_supervised_model.save(MODELS_DIR / f'{best_model_name.lower().replace(" ", "_")}_model.pkl')

# Salvar detector de anomalias
anomaly_detector.save(MODELS_DIR / 'isolation_forest_detector.pkl')

# Salvar feature engineer (scaler)
import joblib
joblib.dump(feature_engineer, MODELS_DIR / 'feature_engineer.pkl')

print(f"\n✓ Modelos salvos em: {MODELS_DIR}")
print("\nArquivos:")
for f in MODELS_DIR.glob('*.pkl'):
    print(f"  - {f.name}")

## 13. Exemplo de Uso em Produção

In [ ]:
def predict_fraud(transaction_data: pd.DataFrame) -> dict:
    """
    Função para predição de fraude em produção.
    
    Args:
        transaction_data: DataFrame com dados da transação
        
    Returns:
        Dicionário com predição e probabilidade
    """
    # Carregar modelos
    model = FraudClassifier.load(MODELS_DIR / f'{best_model_name.lower().replace(" ", "_")}_model.pkl')
    scaler = joblib.load(MODELS_DIR / 'feature_engineer.pkl')
    
    # Processar features
    X = scaler.transform(transaction_data)
    
    # Predição
    proba = model.predict_proba(X)[:, 1]
    pred = (proba >= 0.5).astype(int)
    
    return {
        'is_fraud': bool(pred[0]),
        'fraud_probability': float(proba[0]),
        'risk_level': 'HIGH' if proba[0] > 0.7 else 'MEDIUM' if proba[0] > 0.3 else 'LOW'
    }

# Exemplo de uso
sample_transaction = X_test.iloc[[0]]
result = predict_fraud(sample_transaction)
print("Exemplo de predição:")
print(f"  É fraude: {result['is_fraud']}")
print(f"  Probabilidade: {result['fraud_probability']:.4f}")
print(f"  Nível de risco: {result['risk_level']}")

---

## Resumo do Projeto

Este notebook implementou um sistema completo de detecção de fraude com:

1. **EDA**: Análise exploratória identificando o severo desbalanceamento e padrões nas features
2. **Feature Engineering**: Criação de features temporais, de valor e interações
3. **Modelos Supervisionados**: Treinamento e comparação de 4 algoritmos
4. **Detecção de Anomalias**: Isolation Forest para fraudes emergentes
5. **Sistema Híbrido**: Combinação de abordagens para máxima cobertura
6. **Avaliação**: Métricas apropriadas para dados desbalanceados
7. **Deploy**: Salvamento de modelos prontos para produção

### Próximos Passos
- Implementar API REST para servir predições
- Criar dashboard de monitoramento
- Configurar retreinamento automático